# Variables
Una variable es el nombre asociado a un valor. En la libreta anterior asociamos valores encadenados a un nombre: `str`
Las variables así como las cadenas pueden contener caractéres unicode:

In [ ]:
α² = "alfa cuadrado"

Es importante tener cuidado al utilizar nombres ya utilizados, pues el valor de las variables se sobre-escribe:

In [ ]:
println(sqrt(100))
sqrt = 2

In [ ]:
sqrt(100)

In [ ]:
sqrt = Base.sqrt

In [ ]:
sqrt(2)

Para convenciones y estílos leer:
http://docs.julialang.org/en/release-0.5/manual/variables/

# Tipos
Como ya comentamos, el systema de tipado de Julia es una de las herramientas claves del lenguaje. Aunque no es necesario que le demos un tipo a todo (ya que Julia infiere inteligentemente los tipos) el tipo de cada variable puede ser "anotado" agregandolo después de su nombre y dos "dos puntos" (:)

In [ ]:
1::Int

Este es simplemente un sistema de filtro que indica al compilador cómo debe de tratar a un número o variable. Si el número o variable no corresponden con la anotación, Julia manda un error:

In [ ]:
1.1::Int

### Uniones
También es posible crear uniones de tipos: filtros que dejaran pasar cualquiera de los tipos que pertenescan a su conjunto:

In [ ]:
IntOrFloat = Union{Int,AbstractFloat}

In [ ]:
1::IntOrFloat

In [ ]:
1.1::IntOrFloat

In [ ]:
"1.1"::IntOrFloat

## Abstractos
Para la correcta interacción entre tipos, como hemos visto hasta ahora, es necesario establecer orden entre ellos. Una consecuencia de esto son las relaciones que hay entre los distintos tipos numéricos:
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Type-hierarchy-for-julia-numbers.png/800px-Type-hierarchy-for-julia-numbers.png" alt="Numbers" width="800px">


Éste es el pedacito de Jula en que se declaran los tipos abstractos:

```julia
abstract Number
abstract Real     <: Number
abstract AbstractFloat <: Real
abstract Integer  <: Real
abstract Signed   <: Integer
abstract Unsigned <: Integer
```

## Tipos de distintos bits
Para concretar los tipos numéricos, Julia asigna espacio en bits, como ya hemos visto. Como programador tienes acceso a crear nuevos tipos numéricos si deseas. Éste es el pedacito de Jula en que se declaran los tipo concretos con cada bit:
```julia
bitstype 16 Float16 <: AbstractFloat
bitstype 32 Float32 <: AbstractFloat
bitstype 64 Float64 <: AbstractFloat

bitstype 8  Bool <: Integer
bitstype 32 Char

bitstype 8  Int8     <: Signed
bitstype 8  UInt8    <: Unsigned
bitstype 16 Int16    <: Signed
bitstype 16 UInt16   <: Unsigned
bitstype 32 Int32    <: Signed
bitstype 32 UInt32   <: Unsigned
bitstype 64 Int64    <: Signed
bitstype 64 UInt64   <: Unsigned
bitstype 128 Int128  <: Signed
bitstype 128 UInt128 <: Unsigned
```
[Julia Docs](http://docs.julialang.org/en/release-0.5/manual/types/#bits-types)

## Compuestos
Similar al "Struct" de C, Julia nos permite crear tipos compuestos (composit). Estas son estructuras útiles para la abstracción.

In [ ]:
type Point2d
    x::Int
    y::Int
end
z = Point2d(2,3)

Julia crea automáticamente una función que te permite construir nuevos elementos del tipo que acabas de crear:

In [ ]:
z

In [ ]:
?Point2d

No es necesario que cada tipo compuesto tenga anotación de sus tipos:

In [ ]:
type Foo
    bar
    baz
end

In [ ]:
foo = Foo("Perro",2.2)

Una vez que se ha creado un objeto de tipo compuesto es posible modificar sus partes. Para esto es necesario saber los nombres de cada campo.

In [ ]:
fieldnames(foo)

Se accede a los campos del tipo con el operador `.`

In [ ]:
foo.bar = "Gato"
foo

#### [Inmutables](http://docs.julialang.org/en/release-0.5/manual/types/#immutable-composite-types)
Compuestos inmutables (no se pueden modificar). Se pasan por copia, y pueden ser más eficientes en ciertos casos. 
```julia
immutable Complex
    real::Float64
    imag::Float64
end
```

## Tipos Paramétricos Compuestos
Ya que todos los tipos compestos que hagamos pueden necesitar distintos tipos para sus campos. (Un punto puede estár en cordenadas continuas (x,y) por lo que se necesitarían flotantes para representarlas, o discretas, que requerirían enteros) Julia nos permite crear tipos compuestos que reciben parámetros. Por ejemplo:

In [ ]:
type Point2D{T}
    x::T
    y::T
end

En este caso a la hora de declarar mi objeto, tengo que eviar un tipo como parametro para los campos del mismo:

In [ ]:
Point2D{Float64}(2,3)

In [ ]:
Point2D{AbstractString}("≥","")

De esta forma hemos empezado a crear un árbol de tipos. La raíz de éste árbol es el tipo `Point2D{T}`:

In [ ]:
Point2D

Puedes preguntar a Julia si un objeto es subtipo de otro con lo siguiente notación:

In [ ]:
Point2D{Float64} <: Point2D

## Tipos Paramétricos Abstractos
De la misma manera en que los números en julia pueden formar un árbol de tipos a través de los tipos abstractos, los tipos compuestos pueden tener tipos abstractos, que también pueden recibír parámetros.

In [ ]:
abstract Pointy{T}

In [ ]:
type Point{T} <: Pointy{T}
    x::T
    y::T
end

In [ ]:
Point{Real} <: Pointy{Real} # Point{Real} es subtipo de Pointy{Real}?

In [ ]:
type DiagPoint{T} <: Pointy{T}
    x::T
end

De esta forma, `Point` y `DiagPoint` son subtipos de `Pointy`.

## Funciones útiles
Existen funciones útiles para el manejo de tipos de datos. A continuación presento las más comunes.
- `typeof()` Regresa el tipo concreto de una variable.
- `isa()` Regresa cierto si una variable tiene el tipo o es subtipo del segundo argumento. 
- `supertype()` Regresa el primer supertipo de la variable ingresada.
- `convert()` Convierte una variable a otro tipo.
- `parse()` Procesa texto: puede usarse para convertir texto a números.

In [ ]:
typeof(1//6)

In [ ]:
println(isa(1, Float64))
println(isa(1.0, Float64))

In [ ]:
supertype(Float64)

In [ ]:
convert(Float64, 1)

In [ ]:
parse(Float64, "3.14")

## Convertir un tipo compuesto
Cuando creas un tipo compuesto, a veces es necesario convertir a otros tipos. Para esto puede crearse una función de conversión. 

In [ ]:
convert(::Type{AbstractFloat}, p::Point) = abs(p.x,p.y)

Más adelante hablaremos sobre funciones a profundidad

## Aliases de Tipos
Existen tipos de datos que tinen un alias con el fin de facilitar el llamado de tipos dependientes de bits:
- `Int`
- `UInt`

In [ ]:
println(typeof(Int(2)))
println(typeof(UInt(2)))